# 1. Chủ đề

In [8]:
import sqlite3
import pandas as pd

csv_file = 'categories.csv'

with open(csv_file, 'r') as file:
    df = pd.read_csv(file)
    df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

    df.dropna(inplace=True, axis=0, how='all')

df_processed = df.ffill()
df_processed.rename(columns={'Nhóm nghĩa loại':'category', 'Chủ đề lớn (TV)':'sub_category', 'Chủ đề con (TV)':'sub_sub_category','Unnamed: 3': 'idioms'}, inplace=True)
df_processed = df_processed.map(lambda x: x.strip() if isinstance(x, str) else x)
print(df_processed)

                            category                    sub_category  \
0             Thiên nhiên - Vật chất  Thành ngữ về mười hai con giáp   
1             Thiên nhiên - Vật chất  Thành ngữ về mười hai con giáp   
2             Thiên nhiên - Vật chất  Thành ngữ về mười hai con giáp   
3             Thiên nhiên - Vật chất  Thành ngữ về mười hai con giáp   
4             Thiên nhiên - Vật chất  Thành ngữ về mười hai con giáp   
..                               ...                             ...   
417  Học thuật – Nhận thức – Văn hóa  Thành ngữ về mức độ quan trọng   
418  Học thuật – Nhận thức – Văn hóa  Thành ngữ về mức độ quan trọng   
419  Học thuật – Nhận thức – Văn hóa  Thành ngữ về mức độ quan trọng   
420  Học thuật – Nhận thức – Văn hóa  Thành ngữ về mức độ quan trọng   
421  Học thuật – Nhận thức – Văn hóa  Thành ngữ về mức độ quan trọng   

                            sub_sub_category  \
0                    Thành ngữ về chuột (Tý)   
1                    Thành ngữ về trâu 

Bỏ phần ngăn cách thừa giữa các thành ngữ và chuyển thành danh sách

In [9]:
df_processed.columns

Index(['category', 'sub_category', 'sub_sub_category', 'idioms'], dtype='object')

In [10]:
import re, json

df_processed['idioms'] = df_processed['idioms'].apply(lambda x: re.split(r'[\n\s]+', x.strip()))
df_processed['idioms'] = df_processed['idioms'].apply(lambda x: json.dumps(x, ensure_ascii=False))


In [11]:
df_processed.head()

,category,sub_category,sub_sub_category,idioms
0,Thiên nhiên - Vật chất,Thành ngữ về mười hai con giáp,Thành ngữ về chuột (Tý),"[""三蛇七鼠"", ""两鼠斗穴"", ""以狸至鼠"", ""以狸致鼠"", ""以狸饵鼠"", ""以貍至鼠..."
1,Thiên nhiên - Vật chất,Thành ngữ về mười hai con giáp,Thành ngữ về trâu (Sửu),"[""一牛九锁"", ""一牛吼地"", ""一牛鸣一"", ""一牛鸣地"", ""九牛一毛"", ""九牛一毫..."
2,Thiên nhiên - Vật chất,Thành ngữ về mười hai con giáp,Thành ngữ về hổ (Dần),"[""一虎不河"", ""三人成虎"", ""三言讹虎"", ""与虎添翼"", ""与虎谋皮"", ""两虎相争..."
3,Thiên nhiên - Vật chất,Thành ngữ về mười hai con giáp,Thành ngữ về thỏ (Mão),"[""一雕双兔"", ""东兔西乌"", ""东门逐兔"", ""乌踆兔走"", ""乌飞兔走"", ""兔丝燕麦..."
4,Thiên nhiên - Vật chất,Thành ngữ về mười hai con giáp,Thành ngữ về rồng (Thìn),"[""一龙一猪"", ""乘龙配凤"", ""事火咒龙"", ""云蒸龙变"", ""云起龙襄"", ""人中龙虎..."


In [12]:
df_processed.to_sql('categories', sqlite3.connect('categories.db'), if_exists='replace', index=False)

422

# 2. Thành ngữ

In [13]:
import sqlite3
import pandas as pd

csv_file = 'idioms.csv'

with open(csv_file, 'r') as file:
    df = pd.read_csv(file)
    # df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')

    df.dropna(inplace=True, axis=0, how='all')
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df = df[df['nghia'].notna() & (df['nghia'].str.strip() != '')]
print(df.shape)

(1016, 15)


/tmp/ipykernel_21718/3385503320.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [14]:
import unicodedata

def remove_diacritics(text):
    if pd.isna(text):
        return ""
    return ''.join(
        c for c in unicodedata.normalize('NFD', str(text))
        if unicodedata.category(c) != 'Mn'
    )
df['am_han_viet_ascii'] = df['am_han_viet'].apply(remove_diacritics)

In [15]:
df.head()

,thanh_ngu_tieng_trung,phien_am,am_han_viet,ket_cau_hinh_thuc,ket_cau_ngu_phap,sac_thai_bieu_cam,chu_de,cach_dung,nghia,vi_du_tieng_trung,vi_du_tieng_viet,tu_can_nghia,tu_trai_nghia,nguon_goc_thanh_ngu,cau_chuyen_thanh_ngu,am_han_viet_ascii
0,阿斗太子,ā dǒu tài zǐ,a đẩu thái tử,NaN,Chính phụ,Tiêu cực,Thành ngữ thời Tam Quốc,"Chủ ngữ, Tân ngữ, Định ngữ","Ám chỉ người yếu đuối, kém cỏi, không có khả n...",NaN,NaN,扶不起的阿斗,NaN,"Cao Dương, Hồ Tuyết Nham toàn truyện – Đăng Hỏ...",NaN,a đau thai tu
1,哀兵必胜,āi bīng bì shèng,ai binh tất thắng,NaN,Chủ vị,Tích cực,Thành ngữ về quân sự,Vị ngữ,"Miêu tả tình huống khi một đội quân bị áp bức,...",不，哀兵必胜！不要乐，要哀，何小姐能弹《易水吟》的谱子吗？（《啼笑因缘续集》一０回）,"Không, binh lính mang nỗi buồn mới thắng trận!...",师直为壮,骄兵必败,"Lão Tử – Chương 69: ""Họa không gì lớn hơn khin...",NaN,ai binh tat thang
2,哀哀欲绝,āi āi yù jué,ai ai dục tuyệt,AABC,Bổ sung,Tích cực,Thành ngữ về nỗi đau lòng,"Vị ngữ, Định ngữ, Bổ ngữ",Đau đớn tột cùng; khóc cạn nước mắt,那安太太进门，一眼便看见姑娘，哀哀欲绝的跪在那里……便一直的奔了姑娘去。（清 文康《儿女英雄...,"Bà An vừa bước vào cửa, lập tức trông thấy cô ...",哀痛欲绝、哀天叫地,兴高采烈,"Thanh, Tào Tuyết Cần, Hồng Lâu Mộng – Hồi 13: ...",NaN,ai ai duc tuyet
3,哀声叹气,āi shēng tàn qì,ai thanh thán khí,NaN,Liên hợp,Tiêu cực,NaN,"Vị ngữ, Tân ngữ, Định ngữ",Âm thanh thở dài phát ra do nỗi buồn hoặc đau ...,金玉舟《赵匡胤》第21章：“说罢他仍哀声叹气，愁容不展。”,"Nói xong, ông lại thở dài than thở, nét mặt vẫ...",咳声叹气,喜气洋洋、眉开眼笑,"Minh, Lăng Mông Sơ, Nhị Khắc Phách Án Kinh Kỳ ...",NaN,ai thanh than khi
4,哀思如潮,āi sī rú cháo,ai tư như triều,NaN,Chủ vị,Tích cực,Thành ngữ về nỗi đau lòng,"Vị ngữ, Tân ngữ","Miêu tả nỗi buồn, cảm xúc xót xa dâng trào như...",丈夫遭遇不幸，她哀思如潮,"Chồng gặp bất hạnh, nàng đau buồn như sóng trào.",痛不欲生,欣喜若狂,"Kinh kịch Bình Nguyên Tác Chiến – Hồi 5: ""Triệ...",NaN,ai tu nhu trieu


In [16]:
df.to_sql('idioms', sqlite3.connect('idioms.db'), if_exists='replace', index=False)

1016